In [1]:
#Initial environement of computations


import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal, Temporal
from kanon.calendars import*

Cal=julian_ad = Calendar.registry["Julian A.D."]

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(591, with_units=False)
Hist_table


Years,Entries
IntegerAndSexagesimal,Historical
365 ;,"08 ; 02,45"
730 ;,"16 ; 05,30"
1095 ;,"01s 24 ; 48,29"
1461 ;,"02s 02 ; 51,15"
1826 ;,"02s 10 ; 54,00"
2191 ;,"03s 19 ; 36,59"
2556 ;,"03s 27 ; 39,44"
2922 ;,"04s 05 ; 42,29"
3287 ;,"05s 14 ; 25,28"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(23):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [13]:
#mean motion parameter from the Parisian Alfonsine Tables
daily_mean_elong=Historical(Sexagesimal("12;11,26,41,37,51,50,39"),8)
daily_mean_moon_lat=Historical(Sexagesimal("13;13,45,39,22,25,53,45"),7)
delta_Paris_Toledo=Temporal("0;0,48")
REVO=Historical(Sexagesimal("6,0;0,0"),8)

# computation of the synodic month in DN

synodic_month=REVO/daily_mean_elong
synodic_month=Temporal(synodic_month,8)
synodic_month=synodic_month.decimal
synodic_month=float(synodic_month)


for i in range(23):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=5):

        # Computing the mean sun and mean moon for a given number of synodic years
        months=int(Hist_table[i][0]/synodic_month)
        mean_moon_lat=(months*synodic_month*daily_mean_moon_lat)%REVO
                       
        #filling the Test and Delta tables with the final precision
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*mean_moon_lat
            Delta_recomp[i][1]=60*60*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

Years,Entries
IntegerAndSexagesimal,Historical
365 ;,"08 ; 02,45"
730 ;,"16 ; 05,30"
1095 ;,"01s 24 ; 48,29"
1461 ;,"02s 02 ; 51,15"
1826 ;,"02s 10 ; 54,00"
2191 ;,"03s 19 ; 36,59"
2556 ;,"03s 27 ; 39,44"
2922 ;,"04s 05 ; 42,29"
3287 ;,"05s 14 ; 25,28"


In [14]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(5634.782608695652, nan)

In [15]:
B.astype(int).style.background_gradient(axis=None)

,Entries
Years,
365 ;,0
730 ;,0
1095 ;,0
1461 ;,0
1826 ;,0
2191 ;,0
2556 ;,0
2922 ;,0
3287 ;,0


In [16]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mean syzygy mean arg lat years expanded.csv')
B.to_csv('Mean syzygy mean arg lat years expanded.zip', index=False, compression=compression_opts)